# Positive Chat 


## Obiettivo del progetto
Chi passa qualche ora al PC, soprattutto in giovane età e non per solo scopi lavorativi, sicuramente si è imbattuto in qualche servizio di chating. Spesso tantissime conversazione degenerano in flame e ciò rendono la chat posti particolarmente negativi dove vivere. Soprattutto se ciò che si cerca è solamente un po' di svago e sano confronto di idee. 
L'obiettivo sarà quello di creare delle chat di contenuti e utenti **positivi**. 

## Sviluppo

Per realizzare tutto ciò utilizzerò un sistema di Sentiment Analysis. In particolare la libreria, già testata, adoperata è [VaderSentimentJava](https://github.com/apanimesh061/VaderSentimentJava), ovvero un porting in Java della libreria originale VADER (Valence Aware Dictionary and sEntiment Reasoner). Inoltre, Positive Chat è stato creato con l'utilizzo di potenti servizi e strumenti. 


## Tecnologie Utilizzate

| Servizio | Scopo |  | 
| :------- | :----------- | ----- | 
| **Spring Boot**  | Framework for Reactive and API Data Ingestor| <img src="./imgs/springlogo.svg" width="100" height="60"/> |
| **Apache Kafka** | Data Streamer | <img src="./img/kafkalogo.svg" width="100" height="60"/> |
| **Apache Zookpeer** | Orchestation dei Cluster Kafka | <img src="./img/zookpeerlogo.svg" width="100" height="60"/> |
| **Apache Spark** | Data Processing | <img src="./img/sparklogo.svg" width="100" height="60"/> |
| **Elasticsearch** | Data Indexing | <img src="./img/elasticlogo.svg" width="100" height="60"/> |
| **Kibana** | Data Visualization | <img src="./img/kibanalogo.svg" width="100" height="60"/> |

## Architettura Positive Chat Bot
<img src="./imgs/Architettura-Positive.png"/>

## Spring Boot e Data Ingestor - Positive Ingestor
Spring Boot è un framework open source per scrivere applicazioni enterprise Java. Ha una logica modulare di dipendenze, infatti sono collegati numerosi progetti come Spring Reactive, Spring Data, Spring for Apache Kafka, Spring Integration e tantissimi altri moduli. In questo progetto è stato utilizzato per scrivere delle parti importanti del **Data Ingestor** ed esporre un'API per facilitarne l'utilizzo a dei Client esterni.
Il data ingestor è stato basato su un architettura API di tipo RESTful, riceve tramite i suoi endpoint qualsiasi messaggio, ben formato, che sia di tipo JSON. Questo aspetto permette una scalabilità ampia agli utenti, con la possibilità quindi di adattarsi a qualsiasi piattaforma di chatting e messaggistica che dispongono di API potenti (un esempio è Telegram, ecco perché come esempio vedremo questa piattaforma). Non esclude, quindi, servizi IRC, Slack, Discord, Messenger, Whatsapp, Twitch, YouTube e tanto altro. 

Vediamo gli endpoint tramite degli esempi:

In [13]:
import requests

myRequestBody = {'platform':'telegram', 'userId':'42', 'message':'This is a simply positive message', 'groupId':'33'}
response = requests.post("http://localhost:9000/api/v1/send/telegram-message", json=myRequestBody)
print(response.text)

true


In [12]:
import time

myRequestBody = {'platform':'telegram', 'userId':'42', 'message':'I hate all world and If I were fire, I would burn the world;', 'groupId':'33'}
response = requests.post("http://localhost:9000/api/v1/send/telegram-message", json=myRequestBody)
print(response.text)

time.sleep(20)

response = requests.get("http://localhost:9000/api/v1/getAction/telegram-action")
print(response.text)

true
[{"platform":"telegram","userId":"42","message":"ban","groupId":"33"}]


Il secondo esempio di utilizzo dell'API ha generato un 'action', a seguito del messaggio che è stato inviato. 

Quello che accade è che l'endpoint che sta sul percorso /api/v1/**send**/{topic} produrrà un messaggio nel servizio **Kafka** (che vedremo nel dettaglio più avanti). 
L'endpoint /api/v1/**getAction**/{topic} consumerà il messaggio dal topic **Kafka**, inserendolo in una lista che sarà poi svuotata appena viene letta. 

L'inserimento di uno o più messaggi farà scatenare un evento. Per maggiori approfondimenti è possibile consultare la seguente voce sul [Event-driven architecture](https://en.wikipedia.org/wiki/Event-driven_architecture)


Infine, essendo che VADER fa del sentiment analysis esclusivamente su del testo inglese, al servizio Positive Ingestor è stata demandata la logica di filtrare, ed eventualmente scartare tutti i messaggi che non sono in lingua inglese. Verranno mandati a Kafka solo i messaggi in lingua **inglese**. La libreria utilizzata per fare language detection è questa: [Language Detector](https://github.com/optimaize/language-detector)

In [4]:
from langdetect import detect
import requests

italian = detect("Questo è un semplice testo italiano")
english = detect("This is a simple italian text")
french = detect("Ceci est un texte français simple")
print("italian: " + italian)
print("english: " + english)
print("french: " + french)

myRequestBody = {'platform':'telegram', 'userId':'42', 'message':'Questo è un semplice testo', 'groupId':'33'}
response = requests.post("http://localhost:9000/api/v1/send/telegram-message", json=myRequestBody)
print(response.text)


italian: it
english: en
french: fr
false


## Apache Kafka e Data Streaming

### Message-Oriented Middleware (MOM) 
Si tratta di un'infrastruttura che supporta, orienta, organizza e facilita lo scambio di messaggio di sistemi e servizi distribuiti. Per chiarirci le idee immaginiamo due servizi, separati, che svolgono funzioni diverse, ma che ad un certo punto della loro esecuzione necessitano di scambiarsi informazioni. Come semplice approccio possiamo immaginare che basterebbe aprire una connessione diretta tra i due servizi. Tuttavia si tratta di un problema ricorrente e soprattutto potrebbe diventare un sistema complesso da gestire all'interno dei due servizi. Per questo, risulta spesso molto più utile, gestire la connessione e lo scambio attraverso un servizio terzo (middleware) che si mette in mezzo ai due e facilita l'obiettivo. Allo sviluppatore dei due servizi principali che vogliono comunicare viene sollevata così la responsabilità di gestire la comunicazione. Un implementazione reale di questo concetto viene definito servizio di **Message Broking** (RabbitMQ, **Apache Kafka**). 

Nella maggior parte dei pattern questi due (o più) servizi svolgono il ruolo di Producer e Consumer. Questi due sono accoppiati in maniera debole, cioè che i due servizi non devono conoscersi e nella maggior parte degli scenari neanche si conoscono o conoscono la quantità di Producer e Consumer sono connessi.

<img src="./imgs/mom.png">
Osserviamo il diagramma di sequenza sopra.
Come già detto il MOM si mette in mezzo ai producer e consumer e lo scambio di messaggio. Il Producer produce i messaggi, il Consumer solo quando sarà pronto a farlo consumerà i messaggi: la comunicazione avviene in maniera del tutto **asincrona**; nessuno dei due servizi rimarrà bloccato in attesa. 

### Data Stream
Per data stream si intende una sequenza illimitata (continuamente alimentata) di dati che rappresentano un evento o una serie di eventi. Se pensiamo allo scenario di Positive Chat e proviamo ad immaginare che questo progetto viene ben condiviso dalle community, potremmo aspettarci una quantità di dati da gestire molto complessa e continuamente alimentata. 
Tuttavia, per far funzionare bene il prodotto è necessario che gli utenti malevoli vengono allontanati immediatamente o quasi, è perciò che si parla di **stream processing**, cioè l'analisi di questi dati (che sono sempre e solo messaggi) deve avvenire __on-the-fly__, quindi man mano che i dati sono disponibili devono essere consumati e produrre risultati ogni tot tempo (più  breve possibile, rispettando le capacità computazionali e hardware che abbiamo a disposizione). 

MOM e Streaming Data non sono la stessa cosa: una caratteristica che li distingue notevolmente è che il MOM contiene spesso una risposta, mentre lo Streaming Data consuma una quantità notevole di dati. 
In Positive Chat entrambi pattern sono stati utilizzati.

### Apache Kafka
Apache Kafka è una piattaforma open source di stream processing sviluppata e gestita dalla [Apache Software Foundation](https://www.apache.org/). Ha un vastissimo impiego ed è utilizzata da tantissimi leader del settore. 
Apache Kafka è in grado di pubblicare e ricevere stream di byte di qualsiasi natura. È __fault-tolerant__, e questo lo rende un sistema molto resiliente. Tramite opportune configurazione è possibile renderlo anche __durable__, cioè permette di immagazzinare i record che riceve in maniera permanente. Inoltre, permette di processare i dati in maniera real-time, quindi man mano che li riceve è possibile produrre un risultato. Tuttavia in Positive Chat demandiamo questa responsabilità al servizio di Spark (come alternativa a kSQL) che dopo vedremo.

Come si può facilmente intuire, Apache Kafka, è un servizio esterno che va messo su uno o **più** cluster e anche su più data center. Questo permette di avere anche delle repliche fisiche e sfruttare dei principi di località. Questo però crea degli scenari di gestione più complessi, perché tutti questi cluster di Kafka dovranno essere orchestrati in maniera corretta. Apache Kafka usa come servizio di orchestrazione **Zookpeer** che affronteremo brevemente più avanti. 
Un concetto fondamentale in Kafka sono i **topic** e le **partition**. 


#### Topic
Un topic rappresenta una categoria del record di byte. Immaginiamo nel caso di Positive Chat, abbiamo detto che possono collegarsi diverse tipologia di servizi di messaggistica come datasources, e ogni servizio potrebbe avere il proprio topic dedicato. I Consumatori e Produttori si legano ad uno o più topic. 

#### Partition
Ogni partition è una sequenza di record immutabile e ordinata a cui vengono appesi i dati. Se N consumatori dovranno consumare in broadcast un messaggio dovranno esiste N partizioni e i consumatori dovranno collegarsi 1:1 alle partizioni. Se tutti i consumatori sono collegati ad una singola partizione, il primo disponibile consuma il messaggio. Grazie a questa caratteristica viene applicato anche un meccanismo di load balancing. 


### Apache Zookper e Orchestration
Si tratta di un sistema centralizzato per: 
- mantenere le informazioni e le varie configurazioni (configuration service);
- associare ad ogni servizio un nome (naming registry);
- gestire la sincronizzazione tra i vari servizi (synchronization service). 

Diventa estremamente indispensabile quando un'applicazione services-oriented (o addirittura microservices-oriented) diventa molto complessa e con tanti componenti e copie di servizi da gestire. Apache Kafka ha necessariamente bisogno di Apache Zookpeer, essendo che l'anatomia di Kafka permette, e anzi suggerisce, di avere più cluster replicati (di buona norma, oggigiorno con l'avvento del cloud computing e ancor più degli SaaS è sempre consigliato avere il servizio replicato 3 volte). 
